In [1]:
import pandas as pd
import sqlite3
import numpy as np
# read in all the tables into their own dataframes
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 999
random_seed = 100
np.random.seed(random_seed)

# Path to your SQLite database
database_path = '/home/agartside/CustomJobAlertSystem/DataBase/google_database.db'

# Create a connection to the database
conn = sqlite3.connect(database_path)

# Get a list of all tables in the database
table_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(table_query, conn)

# Create a dictionary to hold dataframes
dataframes = {}

# Load data from each table into a separate DataFrame
for table in tables['name']:
    query = f"SELECT * FROM {table}"
    dataframes[table] = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Now, 'dataframes' is a dictionary with table names as keys and corresponding DataFrames as values
# For example, to access a dataframe, use dataframes['table_name']


In [2]:
# add new column to each dataframe of the job category
for table in tables['name']:
    dataframe = dataframes[table]
    dataframe['Category'] = table[:-5]
    print(dataframe.head())

# Add all the dataframes together into 1 dataframe
job_listings_dataframe = pd.concat([dataframes[table] for table in tables['name']],ignore_index = False)
job_listings_dataframe.head()

job_listings_dataframe = job_listings_dataframe[['JobTitle','Description','Qualifications','Responsibilities','Category']]
job_listings_dataframe = job_listings_dataframe[job_listings_dataframe['Category'] != 'web_developer']
job_listings_dataframe = job_listings_dataframe.sample(frac = 1).reset_index(drop = True)

def remove_whitespace(text):
    return " ".join(text.split())

def clean_data(dataframe):
    # Regex pattern to match various symbols including "•", ",", ".", "\n", "…", "-", and other non-alphanumeric characters
    pattern = r'[•,\n\\…\-\/\(\)\"\!\?\:\;\@\#\$\%\^\&\*\_\+\=\[\]\{\}\|\<\>\`~]'
    for column in dataframe.columns:
        dataframe.loc[:,column] = dataframe[column].astype(str)
        dataframe.loc[:,column] = dataframe[column].str.replace(pattern, ' ', regex = True)
        dataframe.loc[:,column] = dataframe[column].str.lower()
        dataframe.loc[:,column] = dataframe[column].apply(remove_whitespace) 
    return dataframe

job_listings_dataframe.head()

   JobID                                                       ListHashing  \
0      1  87614021101e011842bb7a94c4fc4f554a7d205bbef5fe81f43f7eea3989a540   
1      2  c8f9d9f9ce663ef31a1bc48fdbc33049ef4220e59f1b74f880857fb4c747d463   
2      3  c03b0ac2cbde62a6a000e2f9eb747b957ab12650a6a290b3580f19c351846b1e   
3      5  31bff1a9543a6592ab06d020029d633ff696f7ddf7da84cace06fea1b5f90ee5   
4      6  33cc3f411b485c88b7732c92f91772eba04156bcc51ef3a911e5f1176694f023   

                               CompanyTitle  \
0                             ClearanceJobs   
1                                     Adobe   
2                                      Dice   
3                                Cosmogence   
4  Johns Hopkins Applied Physics Laboratory   

                                                                    JobTitle  \
0         Software Engineer I - (Onsite) Summer 2024 with Security Clearance   
1                                            2024 Intern - Software Engineer   
2       

,JobTitle,Description,Qualifications,Responsibilities,Category
0,Cyber Security Analyst,"Please feel free to submit your resume for future employment considerations. Email all resumes to info@relevanttechnologies.com with the word ""Resume"" in the subject line.\n\nIf you are interested in an internship, download our internship application...\n\nRelevant Technologies currently has a job opening for a Senior Cyber Security Analyst. The position requires 5 years of experience, a Bachelors degree, and expert knowledge of security audit, compliance, and risk assessment practices. Knowledge of NIST Special Publications related to FISMA compliance, a must, DIACAP desirable. Former experience as a Windows or UNIX systems administrator highly desirable. Knowledge of security products such as firewalls, intrusion detection systems, and access control products etc. is highly desired. CFCPs are preferred. CISM or CRISC may be acceptable for the right candidate. Must have excellent writing and communications skills.\n\nThe position is based in Columbia, Maryland and no relocation i...","•The position requires 5 years of experience, a Bachelors degree, and expert knowledge of security audit, compliance, and risk assessment practices •CISM or CRISC may be acceptable for the right candidate •Must have excellent writing and communications skills",None,cybersecurity
1,Cybersecurity & Technology Audit Internship (Summer 2024),"Pay Philosophy The typical starting salary range for this role is determined by a number of factors including skills, experience, education, certifications and location. The full salary range for this role reflects the competitive labor market value for all employees in these positions across the national market and provides an opportunity to progress as employees grow and develop within the... role. Some roles at Liberty Mutual have a corresponding compensation plan which may include commission and/or bonus earnings at rates that vary based on multiple factors set forth in the compensation plan for the role.\n\nDescription A career in finance at Liberty Mutual is more than just balancing assets and liabilities. You'll be using innovative tools and problem-solving skills to fuel your growth and success-and ours. We're dedicated to helping industry-leading finance talent realize their dreams-straight out of college.\n\nSound like you? Read on! The details When you take on the role ...","•You are pursuing a Bachelor's degree in Business Management, Management Information Systems (MIS), Computer Science, or equivalent training, with a minimum 3.0 cumulative GPA •You are detail-oriented and thrive in a fast-paced work environment •You possess strong interpersonal, communication, organizational, and leadership skills •You have experience with technical programs, including a working knowledge of Excel •You are able to commit to a full-time internship over the course of 11-12 weeks","•The details When you take on the role of summer intern with Liberty Mutual's Cybersecurity & Technology Audit department, you're signing on to a collaborative team that's responsible for auditing all of our internal control systems •Your key responsibilities during this rewarding 11-12 internship program will include interviewing personnel, performing testwork, analyzing results, and communicating issues with managers •You'll also participate in audit project teams to complete risk assessments and control evaluations for IT areas under review",cybersecurity
2,Information Technology Intern,"Job Details Level: Entry\nJob Location: Fairfield Location - Fairfield, OH\nSalary Range: Undisclosed...\n\nDescription\n\nThe Application Developer writes and debugs programs for the core database, web server, and network. Performs analysis, design, programming and testing of business applications. Maintains, documents, and provides technical support for systems software, as well as modifying existing and creating new systems software for specific company needs.

In [3]:
print(len(job_listings_dataframe))

1811


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

np.random.seed(random_seed)
X_train, X_test_and_val, y_train, y_test_and_val = train_test_split(job_listings_dataframe['JobTitle'],
                                                                    job_listings_dataframe['Category'],
                                                                    test_size = 0.3,
                                                                    stratify=job_listings_dataframe['Category'])

X_test, X_val, y_test, y_val = train_test_split(X_test_and_val, y_test_and_val, test_size = 0.5, stratify = y_test_and_val)


random_forest_pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                                   ('classifier', RandomForestClassifier(n_jobs=-4))])

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

random_forest_pipeline.fit(X_train, y_train_encoded)


Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('classifier', RandomForestClassifier(n_jobs=-4))])

In [5]:
untuned_predictions = []

In [6]:
random_forest_test_predictions = random_forest_pipeline.predict(X_test)
print(classification_report(y_test_encoded,random_forest_test_predictions))
print(accuracy_score(y_test_encoded, random_forest_test_predictions))
untuned_predictions.append(classification_report(y_test_encoded,random_forest_test_predictions))
untuned_predictions.append(accuracy_score(y_test_encoded, random_forest_test_predictions))

              precision    recall  f1-score   support

           0       0.93      0.84      0.88        50
           1       0.96      0.97      0.97        79
           2       0.84      0.88      0.86        73
           3       0.96      0.97      0.96        70

    accuracy                           0.92       272
   macro avg       0.92      0.92      0.92       272
weighted avg       0.92      0.92      0.92       272

0.9227941176470589


In [7]:
random_forest_val_predictions = random_forest_pipeline.predict(X_val)
print(classification_report(y_val_encoded,random_forest_val_predictions))
print(accuracy_score(y_val_encoded, random_forest_val_predictions))
untuned_predictions.append(classification_report(y_val_encoded,random_forest_val_predictions))
untuned_predictions.append(accuracy_score(y_val_encoded, random_forest_val_predictions))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90        50
           1       0.99      0.97      0.98        79
           2       0.93      0.89      0.91        74
           3       0.90      0.96      0.93        69

    accuracy                           0.93       272
   macro avg       0.93      0.93      0.93       272
weighted avg       0.93      0.93      0.93       272

0.9338235294117647


In [8]:
prediction = random_forest_pipeline.predict(["software engineer"])
print(prediction)
print(label_encoder.inverse_transform(prediction))

[3]
['software_development']


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

np.random.seed(random_seed)

X_train, X_test_and_val, y_train, y_test_and_val = train_test_split(job_listings_dataframe['JobTitle'],
                                                                    job_listings_dataframe['Category'],
                                                                    test_size = 0.3,
                                                                    stratify=job_listings_dataframe['Category'])

X_test, X_val, y_test, y_val = train_test_split(X_test_and_val, y_test_and_val, test_size = 0.5, stratify = y_test_and_val)

# Define the parameter grid for RandomForestClassifier
random_forest_param_grid = {
    'n_estimators': [100, 200, 300, 500, 700, 1000],
    'max_depth': [5, 10, 20, 30, 50, 70, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}


random_forest_grid_search = RandomizedSearchCV(RandomForestClassifier(),
                                                param_distributions = random_forest_param_grid,
                                                n_iter = 500,
                                                cv = 5,
                                                scoring='f1_weighted',
                                                n_jobs = -4)


random_forest_pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                                   ('classifier', random_forest_grid_search)])

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

random_forest_pipeline.fit(X_train, y_train_encoded)


/home/agartside/miniconda3/envs/webscraping/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
905 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
390 fits failed with the following error:
Traceback (most recent call last):
  File "/home/agartside/miniconda3/envs/webscraping/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/agartside/miniconda3/envs/webscraping/lib/python3.11/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/home/agartside/miniconda3/envs/webscraping/lib/python3.11/site-packages/sklearn/bas

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('classifier',
                 RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                                    n_iter=500, n_jobs=-4,
                                    param_distributions={'bootstrap': [True,
                                                                       False],
                                                         'criterion': ['gini',
                                                                       'entropy'],
                                                         'max_depth': [5, 10,
                                                                       20, 30,
                                                                       50, 70,
                                                                       None],
                                                         'max_features': ['auto',
                                                                          'sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': [1,
                                                                              2,
                                                                              4,
                                                                              6,
                                                                              8],
                                                         'min_samples_split': [2,
                                                                               5,
                                                                               10,
                                                                               15,
                                                                               20],
                                                         'n_estimators': [100,
                                                                          200,
                                                                          300,
                                                                          500,
                                                                          700,
                                                                          1000]},
                                    scoring='f1_weighted'))])

In [10]:
random_forest_test_predictions = random_forest_pipeline.predict(X_test)
print(classification_report(y_test_encoded,random_forest_test_predictions))
print(accuracy_score(y_test_encoded, random_forest_test_predictions))
print(untuned_predictions[0])
print(untuned_predictions[1])

              precision    recall  f1-score   support

           0       0.96      0.86      0.91        50
           1       0.96      0.97      0.97        79
           2       0.87      0.89      0.88        73
           3       0.96      0.99      0.97        70

    accuracy                           0.93       272
   macro avg       0.94      0.93      0.93       272
weighted avg       0.93      0.93      0.93       272

0.9338235294117647
              precision    recall  f1-score   support

           0       0.93      0.84      0.88        50
           1       0.96      0.97      0.97        79
           2       0.84      0.88      0.86        73
           3       0.96      0.97      0.96        70

    accuracy                           0.92       272
   macro avg       0.92      0.92      0.92       272
weighted avg       0.92      0.92      0.92       272

0.9227941176470589


In [11]:
random_forest_val_predictions = random_forest_pipeline.predict(X_val)
print(classification_report(y_val_encoded,random_forest_val_predictions))
print(accuracy_score(y_val_encoded, random_forest_val_predictions))
print(untuned_predictions[2])
print(untuned_predictions[3])

              precision    recall  f1-score   support

           0       0.94      0.88      0.91        50
           1       0.97      0.99      0.98        79
           2       0.93      0.91      0.92        74
           3       0.90      0.96      0.93        69

    accuracy                           0.94       272
   macro avg       0.94      0.93      0.93       272
weighted avg       0.94      0.94      0.94       272

0.9375
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        50
           1       0.99      0.97      0.98        79
           2       0.93      0.89      0.91        74
           3       0.90      0.96      0.93        69

    accuracy                           0.93       272
   macro avg       0.93      0.93      0.93       272
weighted avg       0.93      0.93      0.93       272

0.9338235294117647


In [12]:

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(job_listings_dataframe['Category'])

# Fit the pipeline on the full dataset
random_forest_pipeline.fit(job_listings_dataframe['JobTitle'], y_encoded)

# At this point, your pipeline is trained on the entire dataset


/home/agartside/miniconda3/envs/webscraping/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
875 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
420 fits failed with the following error:
Traceback (most recent call last):
  File "/home/agartside/miniconda3/envs/webscraping/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/agartside/miniconda3/envs/webscraping/lib/python3.11/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/home/agartside/miniconda3/envs/webscraping/lib/python3.11/site-packages/sklearn/bas

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('classifier',
                 RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                                    n_iter=500, n_jobs=-4,
                                    param_distributions={'bootstrap': [True,
                                                                       False],
                                                         'criterion': ['gini',
                                                                       'entropy'],
                                                         'max_depth': [5, 10,
                                                                       20, 30,
                                                                       50, 70,
                                                                       None],
                                                         'max_features': ['auto',
                                                                          'sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': [1,
                                                                              2,
                                                                              4,
                                                                              6,
                                                                              8],
                                                         'min_samples_split': [2,
                                                                               5,
                                                                               10,
                                                                               15,
                                                                               20],
                                                         'n_estimators': [100,
                                                                          200,
                                                                          300,
                                                                          500,
                                                                          700,
                                                                          1000]},
                                    scoring='f1_weighted'))])

In [13]:
from joblib import dump

# save model and encoder to file
dump(random_forest_pipeline, 'random_forest_model.joblib')
dump(label_encoder, 'random_forest_encoder.joblib')

['random_forest_encoder.joblib']